In [67]:
using CSV
using DataFrames
using XLSX
using Statistics
using Random

In [68]:
phylum = CSV.read("../processed-data/normalized-otu.csv", DataFrame);

In [69]:
rename!(phylum,:Column1 => :Link_ID);

In [70]:
outcome = CSV.read("../raw-data/metadata.csv", DataFrame)
rename!(outcome,:Column1 => :Link_ID)
outcome = outcome[:, Not(2:10)]
outcome = outcome[:, Not(3:34)]
outcome = outcome[:,1:3];

In [71]:
outcome = filter(row -> !(row.Yield_per_plant == "NA"), outcome);

In [72]:
outcome.Yield_per_plant = parse.(Float64, outcome.Yield_per_plant)
outcome = filter(row -> !(row.Yield_per_plant == 0), outcome);

In [73]:
yield_per_plant = innerjoin(phylum, outcome, on = :Link_ID)
count_binary = innerjoin(phylum, outcome, on = :Link_ID);

In [74]:
sample_size = size(yield_per_plant)[1]
label_idx = size(yield_per_plant)[2]
var_idx = label_idx - 1;

In [75]:
red = zeros(0)
yellow = zeros(0)
russet = zeros(0)
white = zeros(0)
mixed = zeros(0)
red_little = zeros(0)
yellow_little = zeros(0);

for i in 1:sample_size
    if yield_per_plant[i,var_idx] == "RedLittle"
        append!(red_little, yield_per_plant[i, label_idx])
    elseif yield_per_plant[i,var_idx] == "YellowLittle"
        append!(yellow_little, yield_per_plant[i, label_idx])
    elseif yield_per_plant[i,var_idx] == "Red"
        append!(red, yield_per_plant[i, label_idx]) 
    elseif yield_per_plant[i,var_idx] == "Yellow"
        append!(yellow, yield_per_plant[i, label_idx])
    elseif yield_per_plant[i,var_idx] == "Russet"
        append!(russet, yield_per_plant[i, label_idx])
    elseif yield_per_plant[i,var_idx] == "White"
        append!(white, yield_per_plant[i, label_idx])
    else
        append!(mixed, yield_per_plant[i, label_idx])
    end    
end

In [76]:
count_1 = 0
count_0 = 0
# set the label with respect to the variety2
for i in 1:sample_size
    # threshold: 600
    if yield_per_plant[i,var_idx] == "Red"
        if yield_per_plant[i,label_idx] > median(red)
            count_binary[i,label_idx] = 1
        else
            count_binary[i,label_idx] = 0
        end
    elseif yield_per_plant[i,var_idx] == "Russet"
        if yield_per_plant[i,label_idx] > median(russet)
            count_binary[i,label_idx] = 1
        else
            count_binary[i,label_idx] = 0
        end
    elseif yield_per_plant[i,var_idx] == "Yellow"
        if yield_per_plant[i,label_idx] > median(yellow)
            count_binary[i,label_idx] = 1
            count_1 += 1
        else
            count_binary[i,label_idx] = 0
            count_0 += 1
        end
    elseif yield_per_plant[i,var_idx] == "White"
        if yield_per_plant[i,label_idx] > median(white)
            count_binary[i,label_idx] = 1
        else
            count_binary[i,label_idx] = 0
        end
    elseif yield_per_plant[i,var_idx] == "RedLittle"
        if yield_per_plant[i,label_idx] > median(red_little)
            count_binary[i,label_idx] = 1
        else
            count_binary[i,label_idx] = 0
        end
    elseif yield_per_plant[i,var_idx] == "Mixed"
        if yield_per_plant[i,label_idx] > median(mixed)
            count_binary[i,label_idx] = 1
        else
            count_binary[i,label_idx] = 0
        end
    else 
        if yield_per_plant[i,label_idx] > median(yellow_little)
            count_binary[i,label_idx] = 1
        else
            count_binary[i,label_idx] = 0
        end
    end
end
percentage = [(i, count(==(i), count_binary[:,label_idx])) for i in unique(count_binary[:,label_idx])]
println("Percentage of low-yield label: ", round(percentage[1][2] / sample_size, digits = 2))

Percentage of low-yield label: 0.49


In [78]:
count_binary = count_binary[shuffle(1:size(count_binary)[1]), :]

,Link_ID,Actinobacteriota,Myxococcota,Firmicutes,Acidobacteriota,Chloroflexi
,String15,Float64,Float64,Float64,Float64,Float64
1,MN_18_2_Y1,0.198509,0.0292494,0.0663847,0.122088,0.109327
2,MN_1_3_Y1,0.199434,0.039417,0.0290189,0.106983,0.184903
3,MN_25_1_Y1,0.214153,0.0396555,0.0352208,0.183302,0.147439
4,WI_2_3_Y1,0.184209,0.0224645,0.077393,0.158567,0.136376
5,MN_36_3_Y1,0.19925,0.0360383,0.0873866,0.128749,0.107453
6,WI_14_1_Y1,0.179717,0.0708427,0.0589113,0.159955,0.0298285
7,WI_5_3_Y1,0.23768,0.0173427,0.108268,0.142807,0.124704
8,MN_32_3_Y1,0.200098,0.0700415,0.089403,0.0922266,0.0955399
9,MN_21_2_Y1,0.256494,0.0263676,0.0381739,0.125738,0.136954


In [50]:
count_binary = count_binary[:,Not(46)];

In [51]:
CSV.write("../processed-data/binary-data.csv", count_binary)

"../processed-data/binary-data.csv"